**2.1 Mathematical Modelling**  
Let our model variables be x0,x1,x2........x10.  
x0 for course C0  
x1 for course C1  
.  
.  
.  
x10 for course C10

Here each variale takes integer values which is either 0 or 1.  
x[i]=0 : would mean that course Ci is not selected  
x[i]=1 : would mean that course Ci is selected  

x=[x0,x1,x2......,x10]  
credits=[2,3,5,4,6,6,8,8,6,4,6]  

Objective is to maximize total credits.  

max: x' * credits    

such that  
x0+x1+x2+.......+x10>=5  
x0+x1+x2+.......+x10<=9  
x5+x7<=1  
x3+x9<=1  
x7+x2<=1  
x10-x1<=0  
x0+x1==1  
x2+x4<=1  
x8+x10<=1  


In [ ]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

In [3]:
#2.3 solving model using cbc solver
from pyomo.environ import * 
import numpy as np

model=ConcreteModel()

N=11

obj_coef=np.array([2,3,5,4,6,6,8,8,6,4,6])

constr_coef=np.array([1,1,1,1,1,1,1,1,1,1,1])

col_indices=np.arange(N)

model.x=Var(col_indices,domain=Binary)

model.obj=Objective(expr=summation(obj_coef,model.x),sense=maximize)

model.constr1=Constraint(expr=sum(model.x[i]*constr_coef[i] for i in col_indices)<=9)
model.constr2=Constraint(expr=sum(model.x[i]*constr_coef[i] for i in col_indices)>=5)

model.constr3=Constraint(expr=model.x[5]+model.x[7]<=1)
model.constr4=Constraint(expr=model.x[3]+model.x[9]<=1)
model.constr5=Constraint(expr=model.x[7]+model.x[2]<=1)
model.constr6=Constraint(expr=model.x[10]-model.x[1]<=0)
model.constr7=Constraint(expr=model.x[0]+model.x[1]==1)
model.constr9=Constraint(expr=model.x[2]+model.x[4]<=1)
model.constr10=Constraint(expr=model.x[8]+model.x[10]<=1)

model.pprint()

soln=SolverFactory('cbc')

result=soln.solve(model)

print(result)
    
print('\nObjective = ', model.obj())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
          9 :     0 :  None :     1 : False :  True : Binary
         10 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
   

In [8]:
# 2.4 removing restriction that variables are integers
model.x.domain=Reals
model.x.setlb(0)
model.x.setub(1)

model.pprint()

result=soln.solve(model)

print(result)
    
print('\nObjective = ', model.obj())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False :  Reals
          1 :     0 :   1.0 :     1 : False : False :  Reals
          2 :     0 :   0.0 :     1 : False : False :  Reals
          3 :     0 :   0.0 :     1 : False : False :  Reals
          4 :     0 :   1.0 :     1 : False : False :  Reals
          5 :     0 :   0.0 :     1 : False : False :  Reals
          6 :     0 :   1.0 :     1 : False : False :  Reals
          7 :     0 :   1.0 :     1 : False : False :  Reals
          8 :     0 :   0.0 :     1 : False : False :  Reals
          9 :     0 :   1.0 :     1 : False : False :  Reals
         10 :     0 :   1.0 :     1 : False : False :  Reals

1 Objective Declarations
   

**2.5** We can see that the solution obtained by converting MILP to linear programming is exactly the same as that of MILP. So we can say that in this particular case solution of MILP can be obtained by rounding off the solution of the LP, although we don't even need to round off as solution are exactly the same in both cases.

In [12]:
# 2.6 introduction of new course C11 with 9 credits 

model.x.domain=Binary
model.x11=Var(domain=Binary)
model.obj.deactivate()
model.new_obj=Objective(expr=summation(obj_coef,model.x)+9*model.x11,sense=maximize)

model.constr11=Constraint(expr=model.x11+model.x[7]<=1)
model.constr12=Constraint(expr=model.x11+model.x[8]<=1)

model.pprint()

#2.7 solving modified MILP
result=soln.solve(model)

print(result)
    
print('\nObjective = ', model.new_obj())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('x[',11,'] = ', model.x11.value)


    'pyomo.core.base.var.SimpleVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.SimpleVar'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). T

In [13]:
# 2.8 removing restriction that variables are integers
model.x.domain=Reals
model.x.setlb(0)
model.x.setub(1)

model.x11.domain=Reals
model.x11.setlb(0)
model.x11.setub(1)

model.pprint()

result=soln.solve(model)

print(result)
    
print('\nObjective = ', model.new_obj())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('x[',11,'] = ', model.x11.value)

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

2 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False :  Reals
          1 :     0 :   1.0 :     1 : False : False :  Reals
          2 :     0 :   0.0 :     1 : False : False :  Reals
          3 :     0 :   0.0 :     1 : False : False :  Reals
          4 :     0 :   1.0 :     1 : False : False :  Reals
          5 :     0 :   1.0 :     1 : False : False :  Reals
          6 :     0 :   1.0 :     1 : False : False :  Reals
          7 :     0 :   0.0 :     1 : False : False :  Reals
          8 :     0 :   0.0 :     1 : False : False :  Reals
          9 :     0 :   1.0 :     1 : False : False :  Reals
         10 :     0 :   1.0 :     1 : False : False :  Reals
    x11 : Size=1, Index=None


We are getting almost same result from modified MILP and modified LP model except there is some precision error in case of LP. So for this problem solution of the modified MILP can be obtained by rounding off the solution of modified LP.